# Density vs Altitude

Test the best way to incorporate altitude into the a single satellite density dataset. 

Using MSIS (pymsis) modelled density profiles create a density profile from GRACE B observations e.g., scale the MSIS density profile to GRACE B data. 

Two methods will be tested: 

1. Determine the ratio between the density at the altitude of GRACE B. Use this ratio to shift the MSIS density profile to match the observed density at the altitude of GRACE B. 
1. Calculate the scale height from the MSIS model and apply this scale height to GRACE B data to derive an altitude profile. 

Both methods will be compared to CHAMP and GOCE data to determine the best method for deriving a density altitude profile from observations. 

In [18]:
# auto load magic for packages
# as they are being worked on
%load_ext autoreload
%autoreload 2

#print all output in a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
import os

 #add read_io module to current path ()
file_path = 'D:\\GitHub\\DataIO\\'
sys.path.append(os.path.dirname(file_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import data_io
import pandas as pd
import numpy as np

from pymsis import msis
from scipy.optimize import curve_fit

In [ ]:
# define fitting functions
def exp(x, a, b):
    return a * np.exp(-x/b)

def lin(x, a, c):
    return a * x + c

### Read in Data

In [17]:
sdate = '2003-01-01'
edate = '2013-06-01'

# read in grace b data
gr, gr_u, gr_m = data_io.toleos_den.load_toleos(sat='gb',sdate=sdate,edate=edate)



In [ ]:
# read in champ and goce data

ch, ch_u, ch_m = data_io.toleos_den.load_toleos(sat='ch',sdate=sdate,edate=edate)
go, go_u, go_m = data_io.toleos_den.load_toleos(sat='go',sdate=sdate,edate=edate)

In [19]:
gr.head()
gr.tail()

,Date,Time,Time System,alt,lon,lat,lst,arglat,dens_x,dens_mean,flag_0,flag_1,DateTime
0,2003-01-01,00:00:00.000,GPS,496612.671,38.009,-61.572,2.477,241.428,4.082204e-13,5.627000e-13,0.0,0.0,2003-01-01 00:00:00
1,2003-01-01,00:00:10.000,GPS,496812.992,38.016,-62.206,2.480,242.065,4.144426e-13,5.627189e-13,0.0,0.0,2003-01-01 00:00:10
2,2003-01-01,00:00:20.000,GPS,497011.406,38.026,-62.840,2.484,242.701,4.212448e-13,5.627377e-13,0.0,0.0,2003-01-01 00:00:20
3,2003-01-01,00:00:30.000,GPS,497207.827,38.037,-63.474,2.487,243.338,4.248891e-13,5.627566e-13,0.0,0.0,2003-01-01 00:00:30
4,2003-01-01,00:00:40.000,GPS,497402.172,38.051,-64.108,2.491,243.975,4.315686e-13,5.627754e-13,0.0,0.0,2003-01-01 00:00:40


,Date,Time,Time System,alt,lon,lat,lst,arglat,dens_x,dens_mean,flag_0,flag_1,DateTime
29727960,2013-05-31,23:59:10.000,GPS,460624.221,-82.498,-64.594,18.519,295.602,5.208655e-13,8.639601e-13,0.0,0.0,2013-05-31 23:59:10
29727961,2013-05-31,23:59:20.000,GPS,460518.930,-82.482,-63.956,18.523,296.244,5.279037e-13,8.641734e-13,0.0,0.0,2013-05-31 23:59:20
29727962,2013-05-31,23:59:30.000,GPS,460410.142,-82.468,-63.318,18.526,296.885,5.338832e-13,8.643867e-13,0.0,0.0,2013-05-31 23:59:30
29727963,2013-05-31,23:59:40.000,GPS,460297.933,-82.457,-62.679,18.530,297.526,5.401131e-13,8.646000e-13,0.0,0.0,2013-05-31 23:59:40
29727964,2013-05-31,23:59:50.000,GPS,460182.377,-82.448,-62.040,18.533,298.168,5.475876e-13,8.648133e-13,0.0,0.0,2013-05-31 23:59:50


In [12]:
df.head()

,DateTime,Date,Time,Time System,alt,lon,lat,lst,arglat,dens_x,dens_mean,flag_0,flag_1,tt
0,2003-01-01 00:00:00,2003-01-01,00:00:00.000,GPS,496612.671,38.009,-61.572,2.477,241.428,4.082204e-13,5.627000e-13,0.0,0.0,2003-01-01 00:00:00
1,2003-01-01 00:00:10,2003-01-01,00:00:10.000,GPS,496812.992,38.016,-62.206,2.480,242.065,4.144426e-13,5.627189e-13,0.0,0.0,2003-01-01 00:00:10
2,2003-01-01 00:00:20,2003-01-01,00:00:20.000,GPS,497011.406,38.026,-62.840,2.484,242.701,4.212448e-13,5.627377e-13,0.0,0.0,2003-01-01 00:00:20
3,2003-01-01 00:00:30,2003-01-01,00:00:30.000,GPS,497207.827,38.037,-63.474,2.487,243.338,4.248891e-13,5.627566e-13,0.0,0.0,2003-01-01 00:00:30
4,2003-01-01 00:00:40,2003-01-01,00:00:40.000,GPS,497402.172,38.051,-64.108,2.491,243.975,4.315686e-13,5.627754e-13,0.0,0.0,2003-01-01 00:00:40
